# Advent of Code Problem Number 3

> The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

## Part One

In this puzzle, we'll need to analyse a report given in binary numbers to extract the power consumption of the submarine, which is defined by:

$$ p = \gamma \times \epsilon $$

Our gamma rate $\gamma$ is a binary number in which each position of $\gamma$ has the value that is most common in the corresponding position in the report.
Our epsilon rate is calculated in the opposite way, being composed of the least common value of each position.

-----
As always, we'll set up the environment and data first.

In [234]:
from IPython.display import display, Markdown,Latex
import pandas as pd
import numpy as np

input_data:pd.DataFrame = pd.read_csv('../../inputs/3/diagnostic-report.csv',dtype=str)
input_data["report_value"] = input_data["report_value"].str.rstrip()
input_data_split:pd.DataFrame = input_data["report_value"].apply(lambda x: pd.Series(list(x))).astype('int')

With the data in place as columns, we can simply analyse each column to get the value

In [235]:
def most_and_least_common_value_for_column(label,
                                           column: pd.Series,
                                           equal_count_fallback=None,
                                           verbose=False):
    size = column.size
    sum = column.astype('int8').sum()
    average = sum / size

    most_and_least_common_value = (1, 0) if average > 0.5 else (
        0, 1
    ) if average < 0.5 else equal_count_fallback if not equal_count_fallback == None else None

    if most_and_least_common_value == None:
        raise ValueError("No fallback for equal counts")

    if verbose:
        inequality_sign = '>' if average > 0.5 else '<'
        display(
            Latex(f"""
    The average value of column {label} is \(\\frac{{sum}}{{size}}=\\frac{{{sum}}}{{{size}}}={average}\).
    As \({average} {inequality_sign} {0.5}\), the most common value is {most_and_least_common_value[0]}
    """))

    return most_and_least_common_value


def most_and_least_common_values_for_data_frame(data: pd.DataFrame,
                                                verbose=False):
    most_and_least_common_values = []

    for (label, column) in data.iteritems():
        most_and_least_common_values.append(
            most_and_least_common_value_for_column(label,
                                                   column,
                                                   verbose=verbose))

    return most_and_least_common_values


most_and_least_common_values = most_and_least_common_values_for_data_frame(
    input_data_split, verbose=True)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [236]:
gamma_str = ''.join(str(x[0]) for x in most_and_least_common_values)
gamma = int(gamma_str,base=2)
epsilon_str = ''.join(str(x[1]) for x in most_and_least_common_values)
epsilon = int(epsilon_str,base=2)

display(Markdown(f"Finally, the most and least common values of each column are: `{most_and_least_common_values}`"))
display(Latex(f'Therefore, \(\gamma = {gamma_str}={gamma}\) and \(\epsilon = {epsilon_str}={epsilon}\)'))
display(Latex(f'The answer to the puzzle is then \(p=\gamma \\times \epsilon ={gamma}\\times{epsilon}={gamma*epsilon}\)'))

Finally, the most and least common values of each column are: `[(1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0)]`

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Hooray! Correct!

-----

## Part Two

> Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

$ ls = O_2 \times CO_2$

To evaluate $O_2$: Starting at the leftmost position, discard all numbers which do not match the most common value for that position. Continue with the following positions until there's only one left. 
To evaluate $CO_2$: Follow the same process, but use the least common value instead.

The key part to this challenge is continously recomputing the most and least common value.

In [237]:
df_co2 = input_data_split.copy()


def calc_o2(data: pd.DataFrame):
    df_o2 = data.copy()
    for (label, _) in data.iteritems():
        most_common_val, _ = most_and_least_common_value_for_column(
            label, df_o2[label], equal_count_fallback=(1, 0))
        new_df_o2 = df_o2[df_o2[label].eq(most_common_val)]
        if len(new_df_o2.index) >= 1:
            df_o2 = new_df_o2
    return df_o2


def calc_co2(data: pd.DataFrame):
    df_co2 = data.copy()
    for (label, _) in data.iteritems():
        _, least_common_val = most_and_least_common_value_for_column(
            label, df_co2[label], equal_count_fallback=(1, 0))
        new_df_co2 = df_co2[df_co2[label].eq(least_common_val)]
        if len(new_df_co2.index) >= 1:
            df_co2 = new_df_co2
    return df_co2


df_o2: pd.DataFrame = calc_o2(input_data_split)
df_co2: pd.DataFrame = calc_co2(input_data_split)

assert len(df_o2.index) == 1 and len(df_co2.index) == 1

After having done the heavy lifting, we still need to extract the numbers from the dataframes and multiply them.

In [238]:
def convert_result_from_one_line_df(data:pd.DataFrame)->'tuple[str,int]':
  result_arr = data.values[0]
  result_str = ''.join(str(x) for x in result_arr)
  return (result_str,int(result_str,base=2))

co2_str, co2 = convert_result_from_one_line_df(df_co2)
o2_str, o2 = convert_result_from_one_line_df(df_o2)

display(Latex(
f"""
The only values left for \(O_2\) and \(CO_2\) are \({o2_str}\) and \({co2_str}\).

Therefore:

$O_2 = {o2},$
$CO_2 = {co2},$
$ls = O_2 \\times CO_2 = {o2} \\times {co2} = {o2*co2}.$

"""
))

<IPython.core.display.Latex object>

And we're correct again. Great, huh?